In [ ]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

import os

TUTORIAL_DIR = Path(os.getcwd()).as_posix()


***Notebooks are written for Jupyter and might not display well in Gitlab***


# Parameters Identification

In the previous chapters we :
- Cleaned a dataset of measure to feed a physical model of the test bench
- Proposed a physical model of the experimental setup
- Identified the relative influence of the material thermal properties

We concluded that three parameters had a strong influence on the discrepancy between
the temperature measured between two insulation layers and its model prediction.
: the insulation thermal conductivity $\lambda_{etics}$, the coating coefficient
of absorption of solar radiation $\alpha_{coating}$, the thermal resistance $R_{ext}$
modeling external surface conductive and convective heat transfers.

The last parameter is a model assumption, we choose to discard it.
We will only identify the values of $\lambda_{etics}$ and $\alpha_{coating}$

## Identification process

In an identification process, we try to find the values of unknown parameters
that will minimize the gap between a model output and "the ground truth" obtained by measurement.

We hope that the obtained result correspond to the "real value" of the parameters.

We use an optimization algorithm to minimise an error function that describe the
discrepancy between the model output and the measurement.

In this study :
- The optimisation algorithm is <code>differential_evolution</code> from
<code>scipy.optimize</code> (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html)
- The <code>Identificator</code> error function is the $MSE$ describe in the previous chapter and imported
from <code>scikit-learn</code> (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)
*Custom error function will be available in future release*

## Writing the problem using modelitool

### 1- Load the data
First we want to load the measured data we will use for the calibration.
The dataset will be split in two period, one for the calibration process, and one
for the validation.

According to the previous chapters, we chose to use 6 consecutive days, from the
2018-03-22 to the 2018-03-28. Calibration will be performed over 2 days: 22/03 and 23/03,
Validation will be carried on over the remaining period.

We use <code>pandas</code> to load the data

In [ ]:
dataset = pd.read_csv("ressources/study_df.csv",
                      parse_dates=True,
                      index_col=0)

Modelica doesn't understand <code>datetime</code> and uses the number of seconds
since the beginning of the year as time index. We can use <code>modelitool.combitabconvert.datetime_to_seconds</code>
to create a corresponding index.

In [ ]:
from modelitool.combitabconvert import datetime_to_seconds

time_corr = pd.Series(
    datetime_to_seconds(dataset.index),
    index=dataset.index
)

### 1- Load the model
We "import" the modelica model using a <code>Simulator</code>

In [ ]:
from modelitool.simulate import Simulator

In [ ]:
init_dict = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
}

In [ ]:
simulation_opt = {
        "startTime": time_corr.loc["2018-03-22 00:00:00"],
        "stopTime": time_corr.loc["2018-03-23 23:00:00"],
        "stepSize": 300,
        "tolerance": 1e-06,
        "solver": "dassl"
}

In [ ]:
# Values in output list correspond to sensors name and value "T"
simu = Simulator(
    model_path=Path(TUTORIAL_DIR) / "ressources/etics_v0.mo",
    simulation_options=simulation_opt,
    init_parameters=init_dict,
    output_list=["T_ins_ins.T"],
    boundary_df=dataset.loc["2018-03-22 00:00:00":"2018-03-23 23:00:00"]
)

In [ ]:
# These parameters have been identified in a previous study.
simu.set_param_dict({
    'IR_Amb.Gr': 0.06314703438754983,
    'IR_sky.Gr': 0.04675950201241314,
    'C_c.C': 7455.526793655418})

### 2- Set up parameters
Describe the parameters: interval of possible values and initial value.
The names must correspond to Modelica syntax

In our case we estimate that the parameters can't be more or less than 50% of
what we believe to be their "true value"

In [ ]:
id_params = {
    'Lambda_ins.k': {
        "init": 0.04,
        "interval": (0.04*0.4, 0.04*1.6)
    },
    'Alpha_clo.k': {
        "init": 0.5,
        "interval": (0.2, 0.95)
    },
    'R_conv_ext.k': {
        "init": 0.04,
        "interval": (0.02, 0.2)
    },
}

We use a <code>Identificator</code> to put the problem together.
It just requires a <code>Simulator</code> and parameters configuration.
If no <code>error_function</code> is specified, the <code>Identificator</code>
will use the $MSE$ described in the previous chapter.

In [ ]:
from modelitool.identify import Identificator

my_identification = Identificator(
    simulator=simu,
    parameters=id_params,
    error_function=None,
)

the <code>Identificator</code> uses the optimization algorithm to tweak the parameters in order to minimise
the deference between the <code>T_coat_ins.T</code> obtained from the model,
and the specified reference <code>reference_df["T_coat_ins"]</code>

This is done using th <code>fit</code> method, and passing boundary conditions as <code>features</code>
and the reference temperature <code>dataset["T_coat_ins"]</code> as <code>labels</code>

***Careful, this step may require a lot of simulation time***

In [ ]:
feat_train = dataset.loc["2018-03-22 00:00:00":"2018-03-23 23:00:00"]
lab_train = dataset.loc["2018-03-22 00:00:00":"2018-03-23 23:00:00", ["T_Ins_Ins"]] + 273.15

In [ ]:
feat_val = dataset.loc["2018-03-24 00:00:00":"2018-03-28 23:55:00"]
lab_val = dataset.loc["2018-03-24 00:00:00":"2018-03-28 23:55:00", ["T_Ins_Ins"]] + 273.15

In [ ]:
my_identification.fit(
    features=feat_train,
    labels=lab_train
)

Once the optimization process is finished, and if it is considered successful,
we can access the values of the parameters.

In [ ]:
print(my_identification.param_identified)

The results are not satisfying :
- The insulation heat conductivity is set to 0.064, the maximum allowed value. This is very unlikely.
- the solar radiation heat gain $\alpha_{coating}$ is set to the maximum allowed value 0.95
whereas the true value should be around 0.7.

Let's plot the results for the fitting period.
We use <code>predict</code> method to get the results for the desired boundary conditions.

In [ ]:
res_ident_period = my_identification.predict(
    features=dataset.loc["2018-03-22 00:00:00":"2018-03-23 23:00:00"]
)

In [ ]:
import plotly.graph_objects as go

# figure
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = res_ident_period.index,
    y = res_ident_period["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Model_results"
))

fig.add_trace(go.Scatter(
    x = lab_train.index,
    y = lab_train.squeeze(),
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Calibration period Model VS Measure',
    xaxis_title='Time training set',
    yaxis_title='Coating Temperature [K]')

fig.show()

The results are not accurate :
- There seem to be a small "time shift" between real temperature peaks and predicted ones
- The model do ot reproduce the peaks and the quick temperature variation

The algorithm tried to reproduce the high temperatures by maximizing the solar heat gain and the heat transfers
through the first layer of insulation, leading to probably false results

There could be a problem linked to the model heat capacity that have not been considered here.

Let's have a look at the validation period

In [ ]:
res_val_period = my_identification.predict(features=feat_val)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = res_val_period.index,
    y = res_val_period["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Model_results"
))

fig.add_trace(go.Scatter(
    x = lab_val.index,
    y = lab_val.squeeze(),
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Validation period Model / Measure',
    xaxis_title='Time training set',
    yaxis_title='Coating Temperature [K]')

fig.show()

The same previous observations apply to the validation period.

To better understand the error, we propose two metrics :

The Normalized Mean Biased error $NMBE$ :

$
NMBE = \frac{1}{\overline{m}}
\frac
    {\sum \limits_{i=1}^{N} (s_i - m_i)}
    {n}
\times 100
$

The Coefficient of Variation of Root Mean Square Error $CV(RMSE)$ :

$
CV(RMSE) =
\frac{1}{\overline{m}}
\sqrt{
    \frac
        {\sum \limits_{i=1}^{N} (s_i - m_i)^2}
        {n - 1}
}
\times 100
$

With $s$ the simulated values, $m$ the measured values $\overline{m}$ the mean of measured data.

- The metrics are dimensionless. this can be useful to compare the results.
A drawback appears if the mean is to close to 0
- The $NMBE$ characterize the "offset" of the model. Its tendency to over or underestimate the results
- The $CV(RMSE)$ characterize the dispersion between prediction and ground truth. The power $^2$ emphasize
the large errors.

Let's see how the model perform with these 2 metrics for the calibration and validation period.

In [ ]:
from modelitool.metrics import nmbe
from modelitool.metrics import cv_rmse

metrics_df = pd.DataFrame(
    {
        "NMBE" : [
            nmbe(res_ident_period, lab_train),
            nmbe(res_val_period, lab_val)
        ],
        "CVRMSE" : [
            cv_rmse(res_ident_period, lab_train),
            cv_rmse(res_val_period, lab_val)
        ]
    },
    index=["Training", "Validation"]
)

print(metrics_df)

The errors index can be considered as "pretty" good. Despite the probably not so good identification of the physical parameter,
the model is able to accuratly reproduce the wall thermal behavior.

The positive $NMBE$ during both calibration and validation periods confirm that the model have a tendency to overestimate the
insulation temperature

we proppose to perform another calibration using an error function that uses $NMBE$ and $CV(RMSE)$ :

$
error = abs(NMBE) + CV(RMSE)
$

Using the absolute value of $NMBE$ will make the optimization harder to solve. But since the metrics can be negative,
False results will occur when added with $CV(RMSE)$.

_Note that we could add weight to each metric to emphasize their effect_

In [ ]:
def combined_error(y_pred, y_true):
    return abs(nmbe(y_pred,y_true)) + cv_rmse(y_pred,y_true)

In [ ]:
my_combined_identification = Identificator(
    simulator=simu,
    parameters=id_params,
    error_function=combined_error,
)

In [ ]:
my_combined_identification.fit(features=feat_val, labels=lab_val)

In [ ]:
my_combined_identification.param_identified

Oups, the obtained results are completly different.
- $\lambda_{etics}$ is still set to its maximum value
- $\alpha_{coating}$ is low and minimize solar heat gain
- $R_{conv}$ is set to the maximum probably to compensate the lower solar gain

On a physical point of view, these results are a bit less unlikely.
But it is probably also wrong.
Let's compare the two models prediction over the full measurement period

In [ ]:
res_full_custom = my_combined_identification.predict(features=dataset)
res_full = my_identification.predict(features=dataset)

In [ ]:
# figure
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = res_full.index,
    y = res_full["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Model_mse_minimise"
))

fig.add_trace(go.Scatter(
    x = res_full_custom.index,
    y = res_full_custom["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='violet',
    name="Model_custom_minimise"
))

fig.add_trace(go.Scatter(
    x = dataset.index,
    y = dataset.T_Ins_Ins.squeeze() + 273.15,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Calibration period Model VS Measure',
    xaxis_title='Time training set',
    yaxis_title='Coating Temperature [K]')

fig.show()

In [ ]:
from sklearn.metrics import mean_squared_error

metrics_df = pd.DataFrame(
    {
        "NMBE" : [
            nmbe(res_full.squeeze(), dataset.T_Ins_Ins + 273.15),
            nmbe(res_full_custom.squeeze(), dataset.T_Ins_Ins + 273.15)
        ],
        "CVRMSE" : [
            cv_rmse(res_full.squeeze(), dataset.T_Ins_Ins + 273.15),
            cv_rmse(res_full_custom.squeeze(), dataset.T_Ins_Ins + 273.15)
        ],
        "MSE" : [
            mean_squared_error(res_full.squeeze(), dataset.T_Ins_Ins + 273.15),
            mean_squared_error(res_full_custom.squeeze(), dataset.T_Ins_Ins + 273.15)
        ]
    },
    index=["MSE_fitting", "Custom_fitting"]
)

print(metrics_df)

First, the metrics indicate that the calibration process performed well
The new model is better "centered" at the expense of the squared error.

From these information, it is still difficult to draw valid conclusion on the paramters values.

# Conclusion

Model identification is a powerfull tool to obtain material or system properties using measurement
and model.

However, we must keep in mind that a "false" model will provide wrong results.

In this case, we succeeded at creating a model that more or less reproduce the wall thermal behavior.
Unfortunatly we cannot draw any conclusion on the wall material properties. That's a shame as it was the whole point
of the experiment.

Several imporvement can be made to this test bed :
- Perform additionnal measurement such as heat flux
- Provide a more accurate model (better description of phenomenon). But be carefull, this may required more accurate measure such as wind speed and direction.

Finally other identification method (and much more complex and computationnaly intensive) may be used such as stochastic method based on Baye's theorem.
This is the point of the next Chapter __Bayesian Methods for model identification__